## SILVER TO GOLD LAYER

### Gold Layer - Trade History


In [11]:
# Import necessary libraries and utility functions
import pandas as pd
from common.portfolio_calc import Portfolio
from common.utilities import global_path, logger

### Data Processing

- Read and sort trade history data.
- Apply portfolio trade logic.


In [12]:
# Read the CSV file
df_trade_history = pd.read_csv(global_path.tradehistory_silver_file_path)

# Convert 'datetime' to datetime type
df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])

# Sort the DataFrame by 'datetime'
df_trade_history = df_trade_history.sort_values(by="datetime")

logger.info(
    f"Read SILVER Layer trade history data from: {global_path.tradehistory_silver_file_path}"
)

2024-08-07T13:59:44Z - INFO - Read SILVER Layer trade history data from: C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\TradeHistory\TradeHistory_data.csv


### Portfolio Logic Application

- Instantiate Portfolio and apply trade logic.
- Handle expired stocks


In [13]:
# Apply the trade logic to each row of the DataFrame
# Instantiate the Portfolio object
portfolio = Portfolio()
for record in df_trade_history.astype(str).to_dict(orient="records"):
    portfolio.trade(record)

portfolio.check_expired_stocks()
# expired_stocks example : df_trade_history["stock_name"] == "NIFTY-PE-24650-18JUL2024"

2024-08-07T13:59:44Z - INFO - NIFTY-PE-24650-18JUL2024 => 100.0 expired


### Final Processing and Export

- Select and sort relevant columns.
- Save the processed data as a CSV file in the Gold layer.


In [14]:
# Create a DataFrame from the processed data
df_pnl = pd.DataFrame(portfolio.get_pnl())

# Round the values in the columns to two decimal places
df_pnl = df_pnl.round(2)

# Update 'side' column: 'SELL' becomes 'LONG', 'BUY' becomes 'SHORT'
df_pnl["position"] = df_pnl["close_side"].apply(
    lambda x: "LONG" if x == "SELL" else "SHORT" if x == "BUY" else x
)

# Sort the DataFrame by 'segment', 'stock_name', and 'datetime'
df_pnl = df_pnl.sort_values(
    by=["segment", "stock_name", "close_datetime", "open_datetime"]
)

# Round the values in to two decimal places
df_pnl = df_pnl.round(2)

# Reset index to ensure it starts from 0
df_pnl = df_pnl.reset_index(drop=True)

# Select and reorder the columns for the final DataFrame
df_pnl = df_pnl[
    [
        "exchange",
        "segment",
        "stock_name",
        "position",
        "quantity",
        "open_datetime",
        "open_side",
        "open_price",
        "open_amount",
        "close_datetime",
        "close_side",
        "close_price",
        "close_amount",
        "pnl_amount",
        "pnl_percentage",
    ]
]

# Save the final DataFrame to a CSV file
df_pnl.to_csv(global_path.profitloss_gold_file_path, index=None)

logger.info("GOLD Layer CSV file for ProfitLoss successfully created at:")
logger.info(global_path.profitloss_gold_file_path.resolve())

# Display the DataFrame information
df_pnl.info()

2024-08-07T13:59:45Z - INFO - GOLD Layer CSV file for ProfitLoss successfully created at:
2024-08-07T13:59:45Z - INFO - C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\ProfitLoss\ProfitLoss_data.csv


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   exchange        120 non-null    object        
 1   segment         120 non-null    object        
 2   stock_name      120 non-null    object        
 3   position        120 non-null    object        
 4   quantity        120 non-null    float64       
 5   open_datetime   120 non-null    datetime64[ns]
 6   open_side       120 non-null    object        
 7   open_price      120 non-null    float64       
 8   open_amount     120 non-null    float64       
 9   close_datetime  120 non-null    datetime64[ns]
 10  close_side      120 non-null    object        
 11  close_price     120 non-null    float64       
 12  close_amount    120 non-null    float64       
 13  pnl_amount      120 non-null    float64       
 14  pnl_percentage  120 non-null    float64       
dtypes: dat

In [15]:
# Create a DataFrame from the processed data
df_holdings = pd.DataFrame(portfolio.get_holdings())

# Filter the DataFrame to include only rows where the segment is 'EQ' (Equity) or 'MF' (Mutual Funds)
df_holdings = df_holdings[df_holdings["segment"].isin(["EQ", "MF"])]

# Round the values in the columns to two decimal places
df_holdings = df_holdings.round(2)

# Extract date from datetime
df_holdings["date"] = df_holdings["datetime"].dt.date

# Find the index of the maximum datetime for each stock_name and date
idx = df_holdings.groupby(["stock_name", "date"])["datetime"].idxmax()

# Use the indices to filter the original DataFrame
df_holdings = df_holdings.loc[idx].reset_index(drop=True)

# Sort the DataFrame by 'segment', 'stock_name', and 'date'
df_holdings = df_holdings.sort_values(by=["segment", "stock_name", "date"])

# Select and reorder the columns for the final DataFrame
df_holdings = df_holdings[
    [
        "date",
        "segment",
        "exchange",
        "stock_name",
        "holding_quantity",
        "holding_price_avg",
        "holding_amount",
    ]
]

# Save the final DataFrame to a CSV file
df_holdings.to_csv(global_path.holdings_gold_file_path, index=None)

logger.info("GOLD Layer CSV file for Holding successfully created at:")
logger.info(global_path.holdings_gold_file_path.resolve())

# Display the DataFrame information
df_holdings.info()

2024-08-07T13:59:45Z - INFO - GOLD Layer CSV file for Holding successfully created at:
2024-08-07T13:59:45Z - INFO - C:\Users\prashant.tripathi\Code\Upstox\DATA\GOLD\Holdings\Holdings_data.csv


<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, 0 to 28
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               47 non-null     object 
 1   segment            47 non-null     object 
 2   exchange           47 non-null     object 
 3   stock_name         47 non-null     object 
 4   holding_quantity   47 non-null     float64
 5   holding_price_avg  47 non-null     float64
 6   holding_amount     47 non-null     float64
dtypes: float64(3), object(4)
memory usage: 2.9+ KB
